In [14]:
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.layers.recurrent import LSTM,GRU
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
# initializes weights with N(0,0.1)
from keras import initializers
from keras.optimizers import Adam
import keras
from keras.utils import np_utils
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

import sys
#import cPickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
import datetime
import model_ml
import feat_selection as fs


In [21]:
Feat_file = ['Feat_cnt_col_nonmissing','Feat_cnt_tool_nonmissing','Feat_pcent_col_nonmissing','Feat_pcent_tool_nonmissing','All_minus','group_var_std','All_minus_single_col','All_minus_byother_col']
ex_feat = []
for filename in Feat_file:
    if filename == 'Feat_minus_non_missing':
        tmp = pd.read_csv('../../'+filename+'.csv')
    else:
        tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    print(filename)
    print(tmp.shape)

Feat_cnt_col_nonmissing
(721, 3208)
Feat_cnt_tool_nonmissing
(721, 2609)
Feat_pcent_col_nonmissing
(721, 3208)
Feat_pcent_tool_nonmissing
(721, 2609)
All_minus
(721, 4608)
group_var_std
(721, 275)
All_minus_single_col
(721, 5007)
All_minus_byother_col
(721, 3813)


In [15]:



###############
## Load Data ##
###############
print("Load data...")

dfTrain = pd.read_csv('../'+config.original_train_data_path)
dfPred = pd.read_csv('../'+config.original_test_data_path)
# number of train/test samples
num_train, num_test = dfTrain.shape[0], dfPred.shape[0]

print("Done.")


###############
## Load Features ##
###############

print("Load Features")
Feat_file = ['Feat_cnt_col_nonmissing','Feat_cnt_tool_nonmissing','Feat_pcent_col_nonmissing','Feat_pcent_tool_nonmissing','All_minus','group_var_std','All_minus_single_col','All_minus_byother_col']
ex_feat = []
for filename in Feat_file:
    if filename == 'Feat_minus_non_missing':
        tmp = pd.read_csv('../../'+filename+'.csv')
    else:
        tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

print("Done")

###############
## Get dummies ##
###############

print("Get dummies")
predictors = dfTrain.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

dummies_list = ['TOOL_210','TOOL_220','TOOL_300','TOOL_310','TOOL_311','TOOL_312','TOOL_330','TOOL_340','TOOL_344','TOOL_360','TOOL_440','TOOL_520','TOOL_750']

for var in dummies_list:
    #if 'TOOL' in var:
        #continue
    #if dfTrain[var].dtypes=='object':
    predictors.remove(var)
    tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
    tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
    predictors = predictors + tmpTrain.columns.tolist()
    dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
    dfPred = pd.concat([dfPred,tmpPred],axis=1)     
print("Delete the dummy variable missing in dfPred")
for var in predictors:
    if var not in dfPred.columns:
        print(var)
        del dfTrain[var]
        predictors.remove(var)
    
print("Done")


###############
## Feature Selection ##
###############
print("Feature selection")
predictors_rf = fs.RF_selection(dfTrain,predictors)
print('RF Done')
predictors_pear = fs.Pearson_selection(dfTrain,predictors)
print('Pearson Done')
#predictors_mic = fs.MIC_selection(dfTrain,predictors)
#print('MIC Done')


def FeatList(featSlected,bins=5):
    output = [[] for i in range(bins)]
    i = 0
    while i<=len(featSlected)-1:
        for j in range(bins):
            try:
                output[j].append(featSlected[i+j])
            except:
                break
        i += bins
    return output

Load data...
Done.
Load Features
Done
Get dummies
Delete the dummy variable missing in dfPred
TOOL_310_E
TOOL_340_2409.0
TOOL_344_T
TOOL_360_C
Done
Feature selection
RF Done
Pearson Done


In [16]:
featSelected = predictors_rf[:3000]
featNum = len(featSelected)

In [17]:
scale = StandardScaler()
dfTrain.loc[:,featSelected] = scale.fit_transform(dfTrain.loc[:,featSelected])

In [18]:
model = Sequential()
model.add(Dense(int(featNum*1.5),activation='relu',input_shape=(featNum,),kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(featNum,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(int(featNum/2),activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(int(featNum/4),activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(int(featNum/8),activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='adam',loss='mse',metrics=['mae'])


In [20]:
model.fit(dfTrain[featSelected].values,dfTrain['Y'].values,epochs=100,batch_size=5,verbose=2)

Epoch 1/100
55s - loss: 2.6453 - mean_absolute_error: 1.3088
Epoch 2/100


KeyboardInterrupt: 

In [10]:
dfTrain[featSelected].values

array([[ 0.81605198,  0.32343965,  0.18055724, ...,  0.70782362,
         1.51718786, -0.64711363],
       [ 0.78402954,  0.34046638, -0.69945463, ...,  0.70787485,
         0.14204845,  0.16943594],
       [ 0.81605198,  0.3362097 , -0.79806377, ...,  0.70792607,
         1.29046237, -0.64711363],
       ..., 
       [-2.25810299,  0.14465905, -0.64447471, ..., -1.12745493,
        -1.80483351,  0.82730121],
       [ 0.81605198, -1.55801337,  1.22817455, ...,  0.65075931,
        -0.28182965,  1.27890243],
       [-2.25810299,  0.10209224, -0.75443454, ..., -1.1274037 ,
        -0.73035182,  1.09427344]])